## Прект модуля 18 - «Рекомендательные системы»
### Разработка рекомендательной системы для аптек.

In [1]:
import pickle
import string
import annoy

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import notebook
from tqdm import tqdm 
from gensim.models import Word2Vec

import operator

In [2]:
with open("Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)

In [3]:
# checks = pd.read_csv("data.csv",encoding = "ISO-8859-1", engine='python')
checks = pd.read_csv("data.csv")

C:\Users\fadee\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
checks.drop(['name'], axis=1, inplace=True)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00"


In [5]:
# Создадим ключевую колонку: время + касса + магазин
checks['key_col'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)
checks.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
0,2018-12-07,1260627,1455.0,168308.0,906.0,205.0,-1.0,"1,00",2018-12-07_1260627_1455.0
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,"1,00",2018-12-07_198287_279.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,"1,00",2018-12-07_2418385_848.0
3,2018-12-07,1285774,1511.0,168570.0,906.0,205.0,-1.0,"1,00",2018-12-07_1285774_1511.0
4,2018-12-07,1810323,1501.0,168319.0,906.0,205.0,-1.0,"1,00",2018-12-07_1810323_1501.0


#### ОБработаем датасет, удалим нулевые значения, заменим индексы и отсортируем по дате

In [6]:
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)
checks.sample(5)

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
306263,2018-12-06,823777,312.0,28232.0,735.0,170.0,-1.0,"1,00",2018-12-06_823777_312.0
3634992,2018-11-14,553757,308.0,69241.0,718.0,166.0,3016.0,"1,00",2018-11-14_553757_308.0
7891288,2018-03-07,102448,311.0,122384.0,613.0,140.0,-1.0,"1,00",2018-03-07_102448_311.0
17322890,2018-06-08,1055168,348.0,154101.0,395.0,89.0,-1.0,"1,00",2018-06-08_1055168_348.0
19518841,2018-06-29,399746,332.0,51861.0,601.0,135.0,-1.0,"1,00",2018-06-29_399746_332.0


## 1. Создание валидационного датасета

In [7]:
# Достаточно отсортировать и сделать трейн-тест, чтобы построить валидацию.
train, test = train_test_split(checks, test_size=0.3, shuffle=False)

In [8]:
train.tail()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
17307935,2018-06-09,272040,307.0,81948.0,448.0,105.0,-1.0,"1,00",2018-06-09_272040_307.0
17307936,2018-06-09,717152,329.0,63026.0,699.0,162.0,-1.0,"1,00",2018-06-09_717152_329.0
17307937,2018-06-09,373791,317.0,151429.0,738.0,170.0,-1.0,"1,00",2018-06-09_373791_317.0
17307938,2018-06-09,592856,279.0,142296.0,397.0,90.0,-1.0,"1,00",2018-06-09_592856_279.0
17307939,2018-06-09,448173,263.0,73663.0,632.0,146.0,-1.0,"1,00",2018-06-09_448173_263.0


In [9]:
test.head()

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity,key_col
17307933,2018-06-09,176750,313.0,119686.0,395.0,89.0,-1.0,"1,00",2018-06-09_176750_313.0
17307959,2018-06-09,1235404,348.0,52565.0,644.0,148.0,-1.0,"1,00",2018-06-09_1235404_348.0
17307960,2018-06-09,118915,321.0,162842.0,738.0,170.0,-1.0,"1,00",2018-06-09_118915_321.0
17307961,2018-06-09,23784,833.0,89112.0,738.0,170.0,-1.0,"1,00",2018-06-09_23784_833.0
17307981,2018-06-09,812376,869.0,68794.0,445.0,103.0,-1.0,"1,00",2018-06-09_812376_869.0


## 2. Создание контетной рекомендации по названию товаров

In [11]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

sentences = [preprocess_txt(k) for k in product_dict.keys()]

##### Подготовим кандидатогенератор, который будет отдавать похожие продукты 

In [12]:
# Обучим Fasttext и заэмбедим продукты
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_names = annoy.AnnoyIndex(20 ,'angular')

In [13]:
# Будем хранить соответствия не только id-> продукт, но и продукт-> id, чтобы быстрее находить эмбеддинги айтемов
reverse_index_map = {}
counter = 0

for name in notebook.tqdm(product_dict.keys()):
    n_ft = 0
    reverse_index_map[counter] = name
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in preprocess_txt(name):
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_names.add_item(counter, vector_ft)
    counter += 1

ft_index_names.build(10)

  0%|          | 0/30418 [00:00<?, ?it/s]

True

In [14]:
reverse_item_mapping = {v: k for k, v in product_dict.items()}

def recommend(items_list):
    '''Функция создания рекомендации
    по перечню кодов товаров рекомендует 10 похожих товаров'''
    current_vector = np.zeros(20)
    c = len(items_list)+1
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in reverse_index_map:
            c -= 1
            continue
        current_vector += np.array(ft_index_names.get_item_vector[reverse_index_map[iname]])
    return [product_dict[reverse_index_map[i]] for i in ft_index_names.get_nns_by_vector(current_vector / c, 10)]

In [15]:
# Получение контентной рекомендации
recommend(['52539', '217083'])

['(51777) Фестал драже N100 601',
 '(72588) Тампоны спиртовые Soft-Zellin 3 х 6см №100 (9999791) 671',
 '(48005) Тенотен таб. №40 456',
 '(65381) Аторвастатин-Тева табл. п.п.о. 10 мг. №30 685',
 '(37986) Кипферон свечи N10 581',
 '(117208) Ливазо таб.п.п.о.4мг №28 685',
 '(90927) Доктор Столетов Вата мед хирург н/стер 100г 672',
 '(48685) Корега Дентал Вайт отбелив. таб. №30 533',
 '(111024) Пакет "Аптека Озерки" 38 х 50см 906',
 '(53754) Артрозан таб. 15мг № 20 647']

## 3.Создание рекомендации с Word2Vec по чековым данным
#### Обучение эмбеддингов по последовательности

In [16]:
train['product_id'] = train['product_id'].apply(int).apply(str)

grouped = train.groupby('key_col')
sentencesW2V = []

In [17]:
# Сформируем последовательности
sentencesW2V = []
for group in notebook.tqdm(grouped.groups):
    products = grouped.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentencesW2V.append(list(products))

  0%|          | 0/4110156 [00:00<?, ?it/s]

In [18]:
# Строим модель
modelW2V = Word2Vec(sentencesW2V, vector_size=10)

In [19]:
sentencesW2V[0]

['120171', '69028', '62087']

In [20]:
modelW2V.wv.similar_by_vector([1,2])

[('168570', 0.9876341819763184),
 ('217393', 0.9772195816040039),
 ('216024', 0.9749835133552551),
 ('216045', 0.9706698060035706),
 ('217210', 0.9656913876533508),
 ('213615', 0.9606829285621643),
 ('193351', 0.9571182727813721),
 ('192834', 0.9549943804740906),
 ('60795', 0.950454831123352),
 ('192785', 0.9419720768928528)]

In [21]:
def recommend_w2v(items_list):
    '''Функция создания рекомендации
    по перечню кодов товаров рекомендует 10 похожих товаров 
    с использованием эмбедингов по последовательности'''
    current_vector = np.zeros(10)
    c = len(items_list)+1
    for iid in items_list:
        iname = product_dict[iid]
        if iname not in modelW2V.wv:
            c -= 1
            continue
        current_vector += np.array(modelW2V.wv[reverse_index_map[iname]])
    return [product_dict[i[0]] for i in modelW2V.wv.similar_by_vector(current_vector / c, 10)]

In [22]:
# Получение рекомендации с Word2Vec по чековым данным
recommend_w2v(['120171', '69028', '62087'])

['(179474) Маример капли назал.5мл №10 701',
 '(79550) Йода р-р 5% 20мл флакон с кисточкой №1 404',
 '(1386) Виши Деркос Шампунь дерматол. успокаивающий 200мл арт.8081 -1',
 '(38640) Липримар табл. п.о. 10 мг. №30 685',
 '(122307) СВР Себиаклир Крем для лица SPF50 50мл 500',
 '(96684) Редерм мазь д/наруж.прим. туба 15г 467',
 '(12433) Валериана [корневища с корнями пачка 70г] N1 -1',
 '(115414) Солгар Комфорт зон капс.№90 428',
 '(110046454) название -1',
 '(67234) Фитосольба Фитоколор Краска д/волос 8CD Рыжеватый 479']

## 4. Двухстадийная рекомендательная система
####  с кандидатогенерацией и ранжированием по частоте покупки товара

In [23]:
# Создадим метод, который по id продукта будет возвращать похожие на него
def recommend(product_id, num_sampled=10):
    n = 0
    for key, value in reverse_index_map.items():
         if product_id == value:
                n = key          
    if n not in reverse_index_map:
        return []
    return ft_index_names.get_nns_by_item(n, num_sampled)

In [24]:
recommend('214112')

[10117, 19808, 15573, 8360, 3270, 29081, 12608, 8192, 11186, 24769]

In [25]:
# Сделаем сразу рандомный кандидатогенератор (он нам пригодится для кандидатогенерации по субкатегории)
def sample_random(sample_set, num_sampled=10):
    return np.random.choice(sample_set, num_sampled, replace=True)

In [26]:
# Сделаем кандидатогенератор по последним купленным товарам из субкатегории

product2subcat = {}
subcat_dict = {}

for i in notebook.tqdm(train.index):
    subcats = train.loc[i, "product_sub_category_id"]
    product2subcat[train.loc[i, 'product_id']] = subcats
#     for subcat in subcats:
    if subcats not in subcat_dict:
        subcat_dict[subcats] = []
    subcat_dict[subcats].append(train.loc[i, 'product_id'])

  0%|          | 0/14000000 [00:00<?, ?it/s]

In [27]:
# формируем перечень всех купленных товаров
product_unique = train['product_id'].unique()

In [28]:
# Создадим метод, который по последнему купленному товару будет возвращать набор кандидатов
# товаров из субкатегории товаров

def candidate_generator(product_id):

    candidates_ft = []
    if product_id is not None:
        candidates_ft = list(recommend(product_id))
    candidates_random = list(sample_random(product_unique))
    candidates_random = [int(i) for i in candidates_random]
    candidates_ft = [i for i in candidates_ft[1:]]
    candidates_subcat = []
    try:
        for subcat in product2subcat.get(str(product_id), []):
            candidates_subcat += list(sample_random(subcat_dict[subcat], 5))
    except:
        candidates_subcat = sample_random(subcat_dict[product2subcat.get(str(product_id), [])], 5)
    candidates_subcat = [int(i) for i in candidates_subcat]
    # Отфильтруем входной продукт (он мог попасть в кандидаты) и уберем дубликаты
    return set([i for i in candidates_ft + candidates_subcat + candidates_random if i!= product_id])

In [29]:
# Проверим как работает кандидатогенератор
candidate_generator(22453)

{4578,
 13107,
 14380,
 14744,
 19860,
 22703,
 23899,
 24769,
 28168,
 29620,
 36891,
 63322,
 82983,
 87379,
 101264,
 124082,
 128444,
 133041,
 142780,
 154864,
 160686,
 213417,
 218693}

#### Строим ранжирование предлагаемых товаров

кандидатогенерация проходит по субкатегории товара
затем ранжирование из кандидатов по частоте покупки товара

In [31]:
# частота покупки товара
product_freq = train['product_id'].value_counts().to_dict()

In [32]:
def recommend_multy_range(item_list):
    сandidates_list = [] # перечень сгенерированных товаров
    rec_items = [] # перечень рекомендованных товаров
    new_dict = {} # отранжированный словарь рекомендованных товаров 
    
    for iid in item_list:
#       получаем перечень кандидатов
        сandidates_list = list(candidate_generator(iid))
#         ранжируем кандидатов по частоте покупки товара
        for elem in сandidates_list:
            if str(elem) in product_freq.keys():
                new_dict[elem] = product_freq.get(str(elem))
        # сортируем товар по наболее часто покупаемому
    sorted_x = sorted(new_dict.items(), key=operator.itemgetter(1), reverse=True)
#   выдаем 10 наиболее часто покупаемых товаров
    for i in sorted_x[:10]:
        rec_items.append(product_dict[str(i[0])])
    return rec_items

In [34]:
recommend_multy_range(['120171', '69028', '62087'])

['(104090) Эналаприл Гексал таб.10мг №50 738',
 '(38743) Элоком крем 0.1% туба 15г N1 459',
 '(29356) Натекаль ДЗ тб жеват №60 644',
 '(91753) Минеральная вода "Боржоми" 0,5л стекло 569',
 '(59735) Рекицен-РД пор. 100гр. №1 599',
 '(54677) Гевискон сусп. д/приема внутрь (мятная) 150мл 595',
 '(108507) Синафлан мазь 0.025% туба 15г N1 -1',
 '(11217) Прокладки "Molimed premium ultra micro" женск. впитыв. 80 мл. №28 арт.1691319 555',
 '(16281) Прокладки "Molimed premium maxi" женск. впитыв. 790 мл. 14шт. уп. №1 555',
 '(116591) Глюкофаж Лонг таб.пролонг.действ.750мг №30 462']

#### Выводы:
Датасет был разбит на тренировочный и тестовый по вермени чека. 

Посторены контентные рекомендации (по названиям товаров в корзине), а также рекомендации с Word2Vec по чековым данным (обучены эмбеддинги товаров по чековым последовательностям).

Построена двухстадийная рекомендательная система с кандидатогенерацией и ранжированием по частоте покупки товара.

Думаю что для данной модели валидация не имеет смысла, так как товары медицинские и невозможно предсказать по какой причине пришел покупатель, т.е. для лечения каких болезней ему нужны медицинские товары. Поэтому и рекомендации строятся на основе данных о частоте покупки товаров из одной и той же субкатегории. Кроме того, так как покупатель не оцениват товар, то предоположить какой товар может быть куплен можно только по информации из прошлых покупок, т.е. опять, по тому как часто этот товар покупался и вместе с какими товарами.